# Continous Kitsu Scripts
* This notebook will run indefinitely. You must manually kill this notebook once a sufficient number of lists have been found

In [ ]:
import os
import subprocess
import time

from ratelimit import limits, sleep_and_retry

In [ ]:
os.chdir("..")
cwd = os.getcwd()

In [ ]:
procs = {}

In [ ]:
proxyfn = "../../../environment/proxies/proxy_map.txt"
if os.path.exists(proxyfn):
    PROXIES = []
    with open(proxyfn) as f:
        for line in f:
            fields = line.strip().split(",")
            if "kitsu" in fields:
                PROXIES.append(fields[0])
else:
    PROXIES = [0]

In [ ]:
@sleep_and_retry
@limits(calls=1, period=5)
def spawn_process(cmdlist):
    print(cmdlist)
    return subprocess.Popen(cmdlist)

In [ ]:
def spawn_kitsu_process(partition, num_partitions, proxyid):
    cmdlist = [
        "papermill",
        f"{cwd}/GetUserMediaListsKitsu.ipynb",
        "/dev/null",
        "-p",
        "PARTITION",
        str(partition),
        "-p",
        "NUM_PARTITIONS",
        str(num_partitions),
        "-p",
        "PROXY_NUMBER",
        str(proxyid),
    ]
    return spawn_process(cmdlist)

In [ ]:
def spawn(i):
    return spawn_kitsu_process(i, len(PROXIES), PROXIES[i])

In [ ]:
for i in range(len(PROXIES)):
    procs[i] = spawn(i)

In [ ]:
while True:
    time.sleep(1)
    for k, v in procs.items():
        if v.poll() is not None:
            # spawn a new process to replace the finished one
            procs[k] = spawn(k)